In [ ]:
import glob
import shutil
import os
import random
import cv2

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import scikitplot

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from keras.utils import np_utils

from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, ZeroPadding2D, Convolution2D
from tensorflow.keras.layers import Dropout, BatchNormalization, LeakyReLU, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
data = pd.read_csv('/kaggle/input/challenges-in-representation-learning-facial-expression-recognition-challenge/train.csv')

data.head()

In [ ]:
emotion_cat = {0:'anger', 1:'disgust', 2:'fear', 3:'happiness', 4: 'sadness', 5: 'surprise', 6: 'neutral'}

In [ ]:
img = data['pixels'].apply(lambda x: np.array(x.split(' ')).reshape(48, 48, 1).astype('float32'))
img = np.stack(img, axis=0)

le = LabelEncoder()
label = le.fit_transform(data['emotion'])
label = np_utils.to_categorical(label)

print(f'Image Shape: {img.shape}')
print(f'Label Shape: {label.shape}')

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(img, label, shuffle=True, stratify=label, test_size=0.2, random_state=42)

print(f'X training shape: {X_train.shape}')
print(f'X validation shape: {X_valid.shape}')
print(f'Y training shape: {y_train.shape}')
print(f'Y validation shape: {y_valid.shape}')

valid_data = (X_valid, y_valid) 
train_data = (X_train, y_train)

In [ ]:
train_datagen = ImageDataGenerator(
                                    rescale = 1. / 255,
                                    rotation_range=25,
                                    zoom_range=0.15,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.15,
                                    horizontal_flip=True,
                                    fill_mode="nearest")
    
valid_datagen = ImageDataGenerator(
                                    rescale = 1. / 255,
                                    rotation_range=25,
                                    zoom_range=0.15,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.15,
                                    horizontal_flip=True,
                                    fill_mode="nearest")

In [ ]:
BATCH_SIZE = 200
EPOCHS = 100 

train_loader = train_datagen.flow(
        X_train, y_train,
        shuffle=True,
        batch_size=32
    )

valid_loader = train_datagen.flow(
        X_valid, y_valid,
        shuffle=True,
        batch_size=32
    )

In [ ]:
TR_STEPS = len(train_loader)
VA_STEPS = len(valid_loader)

print(TR_STEPS)
print(VA_STEPS)

In [ ]:
np.random.seed(1)
tf.random.set_seed(1)

model = Sequential()

model.add(Conv2D(128, (3,3), activation = 'relu', padding = 'same', input_shape=(48,48,1)))
model.add(Conv2D(128, (3,3), activation = 'relu', padding = 'same'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.30))
model.add(BatchNormalization())

model.add(Conv2D(128, (3,3), activation = 'relu', padding = 'same'))
model.add(Conv2D(128, (3,3), activation = 'relu', padding = 'same'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.30))
model.add(BatchNormalization())

model.add(Conv2D(256, (3,3), activation = 'relu', padding = 'same'))
model.add(Conv2D(256, (3,3), activation = 'relu', padding = 'same'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.30))
model.add(BatchNormalization())

model.add(Conv2D(1024, (3,3), activation = 'relu', padding = 'same'))
model.add(Conv2D(1024, (3,3), activation = 'relu', padding = 'same'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.30))
model.add(BatchNormalization())

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.35))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.35))
model.add(BatchNormalization())

model.add(Dense(7, activation='softmax'))

model.summary()

In [ ]:
optimizer = Adam(learning_rate=0.001)

model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [ ]:
%time 

h1 = model.fit(
        X_train, y_train,
        steps_per_epoch=len(X_train) // BATCH_SIZE,
        validation_data=(X_valid, y_valid),
        validation_steps=len(X_valid) // BATCH_SIZE,
        epochs=EPOCHS
    )

In [ ]:
print("[INFO] Evaluation phase...")

predictions = model.predict(valid_data[0]).argmax(axis=1)

my_classification_report = classification_report(np.argmax(valid_data[1], axis=1), predictions)

print("[INFO] Classification report : ")
print(my_classification_report)

print("[INFO] Total wrong validation predictions : ")
print(np.sum(np.argmax(valid_data[1], axis=1) != predictions))

print("[INFO] Confusion matrix : ")
scikitplot.metrics.plot_confusion_matrix(np.argmax(valid_data[1], axis=1), predictions, figsize=(7,7))

In [ ]:
history = h1.history
n_epochs = len(history['loss'])

plt.figure(figsize=[10,4])
plt.subplot(1,2,1)
plt.plot(range(1, n_epochs+1), history['loss'], label='Training')
plt.plot(range(1, n_epochs+1), history['val_loss'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title('Loss')
plt.legend()
plt.subplot(1,2,2)
plt.plot(range(1, n_epochs+1), history['accuracy'], label='Training')
plt.plot(range(1, n_epochs+1), history['val_accuracy'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.title('Accuracy')
plt.legend()
plt.show()

In [ ]:
tf.keras.backend.set_value(model.optimizer.learning_rate, 0.00001)

In [ ]:
%time 

h2 = model.fit(
        X_train, y_train,
        steps_per_epoch=len(X_train) // BATCH_SIZE,
        validation_data=(X_valid, y_valid),
        validation_steps=len(X_valid) // BATCH_SIZE,
        epochs=EPOCHS
    )

In [ ]:
print("[INFO] Evaluation phase...")

predictions = model.predict(valid_data[0]).argmax(axis=1)

my_classification_report = classification_report(np.argmax(valid_data[1], axis=1), predictions)

print("[INFO] Classification report : ")
print(my_classification_report)

print("[INFO] Total wrong validation predictions : ")
print(np.sum(np.argmax(valid_data[1], axis=1) != predictions))

print("[INFO] Confusion matrix : ")
scikitplot.metrics.plot_confusion_matrix(np.argmax(valid_data[1], axis=1), predictions, figsize=(7,7))

In [ ]:
for k in history.keys():
    history[k] += h2.history[k]

epoch_range = range(1, len(history['loss'])+1)

plt.figure(figsize=[14,4])
plt.subplot(1,2,1)
plt.plot(epoch_range, history['loss'], label='Training')
plt.plot(epoch_range, history['val_loss'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title('Loss')
plt.legend()
plt.subplot(1,2,2)
plt.plot(epoch_range, history['accuracy'], label='Training')
plt.plot(epoch_range, history['val_accuracy'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.title('Accuracy')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
tf.keras.backend.set_value(model.optimizer.learning_rate, 0.0000001)

In [ ]:
%time 

h3 = model.fit(
        X_train, y_train,
        steps_per_epoch=len(X_train) // BATCH_SIZE,
        validation_data=(X_valid, y_valid),
        validation_steps=len(X_valid) // BATCH_SIZE,
        epochs=EPOCHS
    )

In [ ]:
print("[INFO] Evaluation phase...")

predictions = model.predict(valid_data[0]).argmax(axis=1)

my_classification_report = classification_report(np.argmax(valid_data[1], axis=1), predictions)

print("[INFO] Classification report : ")
print(my_classification_report)

print("[INFO] Total wrong validation predictions : ")
print(np.sum(np.argmax(valid_data[1], axis=1) != predictions))

print("[INFO] Confusion matrix : ")
scikitplot.metrics.plot_confusion_matrix(np.argmax(valid_data[1], axis=1), predictions, figsize=(7,7))

In [ ]:
for k in history.keys():
    history[k] += h3.history[k]

epoch_range = range(1, len(history['loss'])+1)

plt.figure(figsize=[14,4])
plt.subplot(1,2,1)
plt.plot(epoch_range, history['loss'], label='Training')
plt.plot(epoch_range, history['val_loss'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title('Loss')
plt.legend()
plt.subplot(1,2,2)
plt.plot(epoch_range, history['accuracy'], label='Training')
plt.plot(epoch_range, history['val_accuracy'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.title('Accuracy')
plt.legend()
plt.tight_layout()
plt.show()